# COVID-19 Dashboard

This dashboard displays the UK data of COVID-19 cases recorded from January 2020 until now. 

In [33]:
from uk_covid19 import Cov19API
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wdg

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

from datetime import datetime

## COVID-19 Cases, Hospital Admissions and Deaths

The following graph displays the daily figures of COVID-19 cases, hospital admissions and deaths in the UK. 
You can CTRL-click to select multiple categories. To get updated data, click on the "Refresh data" button underneath the graph.

In [34]:
def access_api2(): 
    filters = ['areaType=nation',
               'areaName=England']
    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "newDeathsByDeathDate"
    }
    api2 = Cov19API(filters=filters, structure=structure)
    timeseries=api2.get_json()
    #agedistribution=api.get_json()
    
    return timeseries
    #return agedistribution

timedata = {}
with open("timeseries.json", "rt") as INFILE2:
    timedata = json.load(INFILE2)

#print (timedata)

In [36]:
def wrangle_data2(rawdata):
    datalist=timedata['data']
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
   
    def parse_date(datestring):
        return pd.to_datetime(datestring, format='%Y-%m-%d')
    
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])
    
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
            if pd.isna(timeseriesdf.loc[date, column]):
                value = float(entry[column]) if entry[column]!=None else 0.0
                timeseriesdf.loc[date, column] = value
    
    timeseriesdf.fillna(0.0, inplace=True)
    
    return timeseriesdf

timeseriesdf = wrangle_data2(timedata)

#print (timeseriesdf)

In [37]:
#timeseriesdf=pd.read_pickle("timeseriesdf.pkl")

def api_button_callback2(button):
    apidata2=access_api2()
    global df
    df=wrangle_data2(apidata2)
    refresh_graph2()
    apibutton.icon="check"
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Last updated at", current_time)

def refresh_graph2():
    current=series.value[0]
    if current==series.options[0]:
        other=series.options[1]
    else:
        other=series.options[0]
    series.value=(other, ) 
    series.value=(current, ) 
    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='refresh')

series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    value='linear',
    description='Scale:',
    disabled=False)

controls=wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
        plt.title("COVID-19 Cases, Hospital Admissions and Deaths")
        plt.xlabel("Months")
        plt.ylabel("Total Number")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
 
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

apibutton.on_click(api_button_callback2)
display(apibutton)

Output()

Button(description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click to download current Pub…

## COVID-19 Cases by Age and Sex

The following bar chart displays the number of COVID-19 cases by age and sex in the UK.
You can CTRL-click to select multiple categories. To get updated data, click on the "Refresh data" button underneath the chart.

In [38]:
def access_api(): 
    filters = ['areaType=nation',
               'areaName=England']

    structure = {
        "males": "maleCases",
        "females": "femaleCases"
    }
    api = Cov19API(filters=filters, structure=structure)
    agedistribution=api.get_json()
    
    return agedistribution

import json
jsondata = {}
with open("agedistribution.json", "rt") as INFILE:
    jsondata = json.load(INFILE)

#print (jsondata)

In [39]:
def wrangle_data(rawdata):
    datadic=jsondata['data'][0]
    males=datadic['males']
    females=datadic['females']
    ageranges=[x['age'] for x in males] 
    
    def min_age(agerange):
        agerange=agerange.replace('+','') 
        start=agerange.split('_')[0]
        return int(start)
    
    ageranges.sort(key=min_age)
    age_df=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])
    
    for entry in males: 
        ageband=entry['age'] 
        age_df.loc[ageband, 'males']=entry['value']
    
    for entry in females:
        ageband=entry['age']
        age_df.loc[ageband, 'females']=entry['value']
    
    age_df['total']=age_df['males'] + age_df['females']
    
    return age_df

age_df = wrangle_data(jsondata)

In [40]:
def api_button_callback(button):
    apidata=access_api()
    global df
    df=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Last updated at", current_time)

In [30]:
#age_df=pd.read_pickle("agedf.pkl")

agegraph=wdg.SelectMultiple(
    options=['males', 'females', 'total'],
    value=['males', 'females'],
    rows=3,
    description='Stats:',
    disabled=False)

def refresh_graph():
    current=agegraph.value[0]
    if current==agegraph.options[0]:
        other=agegraph.options[1]
    else:
        other=agegraph.options[0]
    agegraph.value=(other, ) 
    agegraph.value=(current, ) 

In [41]:
apibutton2=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='refresh')

def age_graph(gcolumn):
    ncols=len(gcolumn)
    if ncols>0:
        age_df.plot(kind='bar', y=list(gcolumn))
        plt.title("COVID-19 Cases by Age and Sex")
        plt.xlabel("Age Group")
        plt.ylabel("Total Number")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
 
output=wdg.interactive_output(age_graph, {'gcolumn': agegraph})

display(agegraph, output)

apibutton2.on_click(api_button_callback)
display(apibutton2)

SelectMultiple(description='Stats:', index=(0, 1), options=('males', 'females', 'total'), rows=3, value=('male…

Output()

Button(description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click to download current Pub…

**Author and Copyright Notice** 
*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*

[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Joey Fung, 2020 ([ec20174@qmul.ac.uk](mailto:ec20174@qmul.ac.uk)). All rights reserved.